In [20]:
## jsonl
import pandas as pd
import json
import re
# Path to your JSON file
file_path = "/Users/yvette/Desktop/data/part-1-all.jsonl"

# Read the JSONL file line by line
with open(file_path, "r", encoding="utf-8") as file:
    data = [json.loads(line) for line in file]

# Convert to DataFrame
df = pd.json_normalize(data)
print(df.head())

titles_to_remove = ["Back Matter", "Front Matter", "Volume Information", "In Forthcoming Issues","Forthcoming Issues","Note from the Editor","Correction: The Child's Conception of the Queen and the Prime Minister","Correction for: The Length of Ministerial Tenure in the United Kingdom, 1945-97"]
df = df[~df['title'].isin(titles_to_remove) & 
        ~df['docSubType'].isin(['misc', 'news']) &
        ~df['title'].str.contains('Errat', case=False, na=False)]

df.rename(columns={'fullText': 'text'}, inplace=True)
df['creator'] = df['creator'].apply(lambda x: re.sub(r'\[|\]', '', str(x)))
df['creator'] = df['creator'].apply(lambda x: re.sub(r"'", "", str(x)))
df['text'] = df['text'].apply(lambda x: re.sub(r"\[|\]", "", str(x)))
df['text'] = df['text'].apply(lambda x: re.sub(r'"', '', str(x)))
df['text'] = df['text'].apply(lambda x: re.sub(r"'", '', str(x)))
df['keyphrase'] = df['keyphrase'].apply(lambda x: re.sub(r'\[|\]', '', str(x)))
df['keyphrase'] = df['keyphrase'].apply(lambda x: re.sub(r"'", "", str(x)))
df['language'] = df['language'].apply(lambda x: re.sub(r'\[|\]', '', str(x)))
df['language'] = df['language'].apply(lambda x: re.sub(r"'", "", str(x)))


                                            abstract  \
0  How is the unjust global order reproduced, and...   
1                                                NaN   
2  Work exploring the relationship between public...   
3                                                NaN   
4                                                NaN   

                                             creator datePublished  \
0                                 [Anahí Wiedenbrüg]    2021-04-01   
1                                  [R. A. W. Rhodes]    1984-07-01   
2            [Stuart N. Soroka, Christopher Wlezien]    2005-10-01   
3  [Alan Warde, Gindo Tampubolon, Brian Longhurst...    2003-07-01   
4  [Richard M. Merelman, R. W. Connell, Herbert H...    1973-02-01   

         docSubType  docType  \
0  research-article  article   
1  research-article  article   
2  research-article  article   
3  research-article  article   
4       book-review  article   

                                            fullT

In [ ]:
## APS 783 pdf to txt
import os
import fitz  # PyMuPDF
from tqdm import tqdm

# input folder and output folder
input_folder = "/Users/yvette/Desktop/data/APS 783"  
output_folder = "/Users/yvette/Desktop/data/APS 783 txt"  

# make sure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# get all pdf files in the input folder
pdf_files = [f for f in os.listdir(input_folder) if f.endswith(".pdf")]

def pdf_to_text(pdf_path):
    text = []
    doc = fitz.open(pdf_path)
    for page in doc:
        page_text = page.get_text("text").strip()  
        text.append(page_text)
    return "\n".join(text)

# iterate through all pdf files and convert them to text
for pdf_file in tqdm(pdf_files, desc="Converting PDFs to TXT"):
    pdf_path = os.path.join(input_folder, pdf_file)
    text = pdf_to_text(pdf_path)
    
    # generate the output text file name
    txt_filename = os.path.splitext(pdf_file)[0] + ".txt"
    txt_path = os.path.join(output_folder, txt_filename)
    
    # write the text to a file
    with open(txt_path, "w", encoding="utf-8") as txt_file:
        txt_file.write(text)

print(f"所有 PDF 已转换为 TXT，保存在 {output_folder} 文件夹。")


In [ ]:
## APS 783 txt combined with metadata
import os
import pandas as pd
import openpyxl
import re
import urllib.parse
from rapidfuzz import process, fuzz

# Input and output paths
input_folder = "/Users/yvette/Desktop/data/APS 783 txt"  # Your folder containing TXT files

data = []

# Process each TXT file
for txt_file in os.listdir(input_folder):
    if txt_file.endswith(".txt"):
        file_path = os.path.join(input_folder, txt_file)
        
        # Extract the file name (remove .txt)
        filename_without_ext = os.path.splitext(txt_file)[0]

        # **Improve title extraction** (remove year and author)
        match = re.search(r"\d{4} [A-Za-z]+ (.+)", filename_without_ext)
        if match:
            title = match.group(1).strip()  # Only extract article title
        else:
            title = filename_without_ext.strip()  # Keep full file name if no match
        
        # Read the TXT file content
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()  # Read the entire file, remove extra spaces

        # Add the title and content to the data list
        data.append([title, content])  # Store as a list with title and content

# Convert the list of data to a DataFrame
APS_783 = pd.DataFrame(data, columns=["title", "text"])
APS_783['title'] = APS_783['title'].apply(lambda x: urllib.parse.unquote(x))
print(APS_783.head())

file_path = "/Users/yvette/Desktop/data/APS-portico-metadata.csv"
df_csv = pd.read_csv(file_path)

# Remove unwanted rows
df_filtered = df_csv[~df_csv['title'].isin(['Issue Information ‐ Table of Contents', 'Issue Information'])]

print(df_filtered.shape)

def find_best_match(title, choices):
    match, score, index = process.extractOne(title, choices, scorer=fuzz.ratio)  # Use fuzz.ratio for string similarity
    return match if score > 63 else None  


df_filtered["matched_title"] = df_filtered["title"].apply(lambda x: find_best_match(x, APS_783["title"].tolist()))

print(df_filtered.head())
print(df_filtered.shape)
print(df_filtered[df_filtered["matched_title"].isna()])

# Merge the filtered DataFrame with the APS_783 DataFrame
df_merged_APS_783 = df_filtered.merge(APS_783[["title", "text"]], left_on="matched_title", right_on="title", how="left")
df_merged_APS_783 = df_merged_APS_783.drop(columns=["matched_title", "title_y"]).rename(columns={"title_x": "title"})

print(df_merged_APS_783.shape)
print(df_merged_APS_783.head())


In [ ]:
## BPS 909 pdf to txt
import os
import fitz  # PyMuPDF
from tqdm import tqdm

# input folder and output folder
input_folder = "/Users/yvette/Desktop/data/BPS 909"  
output_folder = "/Users/yvette/Desktop/data/BPS 909 txt"  

# make sure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# get all pdf files in the input folder
pdf_files = [f for f in os.listdir(input_folder) if f.endswith(".pdf")]

def pdf_to_text(pdf_path):
    text = []
    doc = fitz.open(pdf_path)
    for page in doc:
        page_text = page.get_text("text").strip()  
        text.append(page_text)
    return "\n".join(text)

# iterate through all pdf files and convert them to text
for pdf_file in tqdm(pdf_files, desc="Converting PDFs to TXT"):
    pdf_path = os.path.join(input_folder, pdf_file)
    text = pdf_to_text(pdf_path)
    
    # generate the output text file name
    txt_filename = os.path.splitext(pdf_file)[0] + ".txt"
    txt_path = os.path.join(output_folder, txt_filename)
    
    # write the text to a file
    with open(txt_path, "w", encoding="utf-8") as txt_file:
        txt_file.write(text)

print(f"所有 PDF 已转换为 TXT，保存在 {output_folder} 文件夹。")


In [ ]:
## BPS 909 txt combined with metadata
import os
import pandas as pd
import re
import openpyxl

# Input and output paths
input_folder = "/Users/yvette/Desktop/data/BPS 909 txt" # Your folder containing TXT files

data = []

# Process each TXT file
for txt_file in os.listdir(input_folder):
    if txt_file.endswith(".txt"):
        file_path = os.path.join(input_folder, txt_file)
        
        # Extract the file name (remove .txt)
        filename_without_ext = os.path.splitext(txt_file)[0]

        # Set title as the filename without extension
        title = filename_without_ext.replace("-", " ")
        
        # Read the TXT file content
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
        # Add the title and content to the data list
        data.append([title, content])  # Store as a list with title and content

# Convert the list of data to a DataFrame
BPS_909 = pd.DataFrame(data, columns=["title", "text"])

# Read the Excel file
file_path = "/Users/yvette/Desktop/data/BPS-portico-metadata.xlsx"
wb = openpyxl.load_workbook(file_path)
ws = wb.active  # 选择第一个工作表

# find the index of the "title" column
header = [cell.value for cell in next(ws.iter_rows(min_row=1, max_row=1))]
title_col_index = header.index("title")

# read the rows and filter out red font color
rows_to_keep = []
for row in ws.iter_rows():
    cell = row[title_col_index] 
    if cell.font.color is None or (cell.font.color.rgb != "FFFF0000"):  
        rows_to_keep.append([c.value for c in row])

# Convert to DataFrame
df_excel = pd.DataFrame(rows_to_keep[1:], columns=rows_to_keep[0])  
df_excel["title"] = df_excel["title"].str.lower().str.replace(r"[:,.!?'-()]", "", regex=True)  

def find_best_match(title, choices):
    match, score, index = process.extractOne(title, choices, scorer=fuzz.ratio)  # Use fuzz.ratio for string similarity
    return match if score > 60 else None  


df_excel["matched_title"] = df_excel["title"].apply(lambda x: find_best_match(x, BPS_909["title"].tolist()))

print(df_excel[df_excel["matched_title"].isna()])


# Merge the filtered DataFrame with the BPS_909 DataFrame
df_merged_BPS_909 = df_excel.merge(BPS_909[["title", "text"]], left_on="matched_title", right_on="title", how="left")
df_merged_BPS_909 = df_merged_BPS_909.drop(columns=["matched_title", "title_y"]).rename(columns={"title_x": "title"})


In [ ]:
# APS 60 pdf to txt
import os
import fitz  # PyMuPDF
from tqdm import tqdm

# input folder and output folder
input_folder = "/Users/yvette/Desktop/data/APS all 60"  
output_folder = "/Users/yvette/Desktop/data/APS all 60 txt"  

# make sure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# get all pdf files in the input folder
pdf_files = [f for f in os.listdir(input_folder) if f.endswith(".pdf")]

def pdf_to_text(pdf_path):
    """将 PDF 转换为干净的文本格式"""
    text = []
    doc = fitz.open(pdf_path)
    for page in doc:
        page_text = page.get_text("text").strip()  
        text.append(page_text)
    return "\n".join(text)

# iterate through all pdf files and convert them to text
for pdf_file in tqdm(pdf_files, desc="Converting PDFs to TXT"):
    pdf_path = os.path.join(input_folder, pdf_file)
    text = pdf_to_text(pdf_path)
    
    # generate the output text file name
    txt_filename = os.path.splitext(pdf_file)[0] + ".txt"
    txt_path = os.path.join(output_folder, txt_filename)
    
    # write the text to a file
    with open(txt_path, "w", encoding="utf-8") as txt_file:
        txt_file.write(text)

print(f"所有 PDF 已转换为 TXT，保存在 {output_folder} 文件夹。")

In [ ]:
# BPS 47 pdf to txt
# input folder and output folder
input_folder = "/Users/yvette/Desktop/data/BPS all 47"  
output_folder = "/Users/yvette/Desktop/data/BPS all 47 txt"  

# make sure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# get all pdf files in the input folder
pdf_files = [f for f in os.listdir(input_folder) if f.endswith(".pdf")]

def pdf_to_text(pdf_path):
    """将 PDF 转换为干净的文本格式"""
    text = []
    doc = fitz.open(pdf_path)
    for page in doc:
        page_text = page.get_text("text").strip() 
        text.append(page_text)
    return "\n".join(text)

# iterate through all pdf files and convert them to text
for pdf_file in tqdm(pdf_files, desc="Converting PDFs to TXT"):
    pdf_path = os.path.join(input_folder, pdf_file)
    text = pdf_to_text(pdf_path)
    
    # generate the output text file name
    txt_filename = os.path.splitext(pdf_file)[0] + ".txt"
    txt_path = os.path.join(output_folder, txt_filename)
    
    # write the text to a file
    with open(txt_path, "w", encoding="utf-8") as txt_file:
        txt_file.write(text)

print(f"所有 PDF 已转换为 TXT，保存在 {output_folder} 文件夹。")

Converting PDFs to TXT: 100%|██████████| 47/47 [00:02<00:00, 18.62it/s]

所有 PDF 已转换为 TXT，保存在 /Users/yvette/Desktop/data/BPS all 47 txt 文件夹。


In [ ]:
# APS 60, BPS 47 merge with metadata
import os
import pandas as pd
import openpyxl
import re
import urllib.parse
from rapidfuzz import process, fuzz

# Input and output paths
input_folder = "/Users/yvette/Desktop/data/APS all 60 txt"  # Your folder containing TXT files

data = []

# Process each TXT file
for txt_file in os.listdir(input_folder):
    if txt_file.endswith(".txt"):
        file_path = os.path.join(input_folder, txt_file)
        
        # Extract the file name (remove .txt)
        filename_without_ext = os.path.splitext(txt_file)[0]

        # **Improve title extraction** (remove year and author)
        match = re.search(r"\d{4} [A-Za-z]+ (.+)", filename_without_ext)
        if match:
            title = match.group(1).strip()  # Only extract article title
        else:
            title = filename_without_ext.strip()  # Keep full file name if no match
        
        # Read the TXT file content
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()  # Read the entire file, remove extra spaces

        # Add the title and content to the data list
        data.append([title, content])  # Store as a list with title and content

# Convert the list of data to a DataFrame
APS_60 = pd.DataFrame(data, columns=["title", "text"])
APS_60['title'] = APS_60['title'].apply(lambda x: urllib.parse.unquote(x))
print(APS_60.head())

# Input and output paths
input_folder = "/Users/yvette/Desktop/data/BPS all 47 txt" # Your folder containing TXT files

data = []
# Process each TXT file
for txt_file in os.listdir(input_folder):
    if txt_file.endswith(".txt"):
        file_path = os.path.join(input_folder, txt_file)
        
        # Extract the file name (remove .txt)
        filename_without_ext = os.path.splitext(txt_file)[0]

        # **Improve title extraction** (remove year and author)
        match = re.search(r"\d{4} [A-Za-z]+ (.+)", filename_without_ext)
        if match:
            title = match.group(1).strip()  # Only extract article title
        else:
            title = filename_without_ext.strip()  # Keep full file name if no match
        
        # Read the TXT file content
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()  # Read the entire file, remove extra spaces

        # Add the title and content to the data list
        data.append([title, content])  # Store as a list with title and content
# Convert the list of data to a DataFrame
BPS_47 = pd.DataFrame(data, columns=["title", "text"])

merged_60_47 = pd.concat([APS_60, BPS_47], ignore_index=True)

# Read the Excel file
file_path = "/Users/yvette/Desktop/data/filtered-all-portico-metadata.xlsx"
wb = openpyxl.load_workbook(file_path)
ws = wb.active  

# find the index of the "title" column
header = [cell.value for cell in next(ws.iter_rows(min_row=1, max_row=1))]
title_col_index = header.index("title")

# read the rows and filter out red font color
rows_to_keep = []
for row in ws.iter_rows():
    cell = row[title_col_index]  
    if cell.font.color is None or (cell.font.color.rgb != "FFFF0000"):  
        rows_to_keep.append([c.value for c in row])

# Convert to DataFrame
df_excel = pd.DataFrame(rows_to_keep[1:], columns=rows_to_keep[0])  
df_excel["title"] = df_excel["title"].str.lower().str.replace(r"[:,.!?'-()]", "", regex=True)  

def find_best_match(title, choices):
    match, score, index = process.extractOne(title, choices, scorer=fuzz.ratio)  # Use fuzz.ratio for string similarity
    return match if score > 60 else None  

df_excel["matched_title"] = df_excel["title"].apply(lambda x: find_best_match(x, merged_60_47["title"].tolist()))

print(df_excel[df_excel["matched_title"].isna()])


# Merge the filtered DataFrame with the merged_60_47 DataFrame
df_merged_60_47 = df_excel.merge(merged_60_47[["title", "text"]], left_on="matched_title", right_on="title", how="left")
df_merged_60_47 = df_merged_60_47.drop(columns=["matched_title", "title_y"]).rename(columns={"title_x": "title"})



                                               title  \
0  Aid  Attitudes  and Insurgency  Evidence from ...   
1  When Should We Use Unit Fixed Effects Regressi...   
2   Legislative Capacity and Executive Unilateralism   
3  The Problem of Political Science  Political Re...   
4              When Governments Regulate Governments   

                                                text  
0  Aid, Attitudes, and Insurgency: Evidence from\...  
1  When Should We Use Unit Fixed Effects Regressi...  
2  Legislative Capacity and Executive Unilaterali...  
3  The Problem of Political Science: Political Re...  
4  When Governments Regulate Governments\nDavid M...  
Empty DataFrame
Columns: [id, title, isPartOf, publicationYear, doi, docType, docSubType, provider, collection, datePublished, issueNumber, volumeNumber, url, creator, publisher, language, pageStart, pageEnd, placeOfPublication, keyphrase, wordCount, pageCount, outputFormat, matched_title]
Index: []

[0 rows x 24 columns]


In [24]:
## ALL together
print(df_merged_APS_783.columns)
print(df_merged_BPS_909.columns)
print(df_merged_60_47.columns)    
print(df.columns)
# Remove specific columns from df_merged_APS_783, df_merged_BPS_909, df
df_merged_APS_783 = df_merged_APS_783.drop(columns=['doi', 'collection', 'placeOfPublication'])
df_merged_BPS_909 = df_merged_BPS_909.drop(columns=['doi', 'collection', 'placeOfPublication'])
df_merged_60_47 = df_merged_60_47.drop(columns=['doi', 'collection', 'placeOfPublication'])
df = df.drop(columns=['abstract', 'identifier', 'pagination', 'sourceCategory', 'tdmCategory'])
df_all = pd.concat([df_merged_APS_783, df_merged_BPS_909,df_merged_60_47, df], ignore_index=True)

Index(['id', 'title', 'isPartOf', 'publicationYear', 'doi', 'docType',
       'docSubType', 'provider', 'collection', 'datePublished', 'issueNumber',
       'volumeNumber', 'url', 'creator', 'publisher', 'language', 'pageStart',
       'pageEnd', 'placeOfPublication', 'keyphrase', 'wordCount', 'pageCount',
       'outputFormat', 'text'],
      dtype='object')
Index(['id', 'title', 'isPartOf', 'publicationYear', 'doi', 'docType',
       'docSubType', 'provider', 'collection', 'datePublished', 'issueNumber',
       'volumeNumber', 'url', 'creator', 'publisher', 'language', 'pageStart',
       'pageEnd', 'placeOfPublication', 'keyphrase', 'wordCount', 'pageCount',
       'outputFormat', 'text'],
      dtype='object')
Index(['id', 'title', 'isPartOf', 'publicationYear', 'doi', 'docType',
       'docSubType', 'provider', 'collection', 'datePublished', 'issueNumber',
       'volumeNumber', 'url', 'creator', 'publisher', 'language', 'pageStart',
       'pageEnd', 'placeOfPublication', 'keyphr

In [25]:
# Replace "Midwest Journal of Political Science" with "American Journal of Political Science"
df_all['isPartOf'] = df_all['isPartOf'].replace('Midwest Journal of Political Science', 'American Journal of Political Science')

# Define the year ranges
year_ranges = ['1971-1980', '1981-1990', '1991-2000', '2001-2010', '2011-2020', '2021-2024']

# Create a function to categorize the year into the specified ranges
def categorize_year(year):
    if 1971 <= year <= 1980:
        return '1971-1980'
    elif 1981 <= year <= 1990:
        return '1981-1990'
    elif 1991 <= year <= 2000:
        return '1991-2000'
    elif 2001 <= year <= 2010:
        return '2001-2010'
    elif 2011 <= year <= 2020:
        return '2011-2020'
    elif 2021 <= year <= 2024:
        return '2021-2024'
    else:
        return None  # For any other years outside this range

# Apply the function to create a new column 'year_range'
df_all['year_range'] = df_all['publicationYear'].apply(categorize_year)


In [26]:
# Aggregate both article counts and word count sums
result = df_all.groupby(["year_range", "isPartOf"]).agg(
    article_count=("isPartOf", "count"), word_count_total=("wordCount", "sum")
).unstack(fill_value=0)

# Flatten column names
result.columns = ["APS_articles", "BPS_articles", "APS_wordcount", "BPS_wordcount"]

# Reset index to include year column
result = result.reset_index().rename(columns={"year_range": "year"})

# Print result
print(result)

        year  APS_articles  BPS_articles  APS_wordcount  BPS_wordcount
0  1971-1980           472           325        2919923        2559913
1  1981-1990           435           264        3977142        2490315
2  1991-2000           538           279        5114556        2828226
3  2001-2010           593           529        6944718        5812471
4  2011-2020          1242           823       14231219        9934559
5  2021-2024           360           314        4419635        3383388


In [27]:
df_all.to_json('df_decade.jsonl', orient='records', lines=True)

In [28]:
### ngram jsonl 8144
import dask.dataframe as dd

file_path = "/Users/yvette/Desktop/data/8144 ngram.jsonl"

ngram = dd.read_json(file_path, lines=True)

print(ngram.head())


                                             creator datePublished  \
0  [Colin Rallings, Michael Laver, Michael Thrasher]    1987-10-01   
1           [Anthony J. McGann, Eliora Van der Hout]    2009-05-15   
2                             [Johannes T. Pedersen]    1978-02-01   
3                     [M. Martin Boyer, André Blais]    1996-04-01   
4                               [Rikhil R. Bhavnani]    2015-11-23   

           docSubType  docType                        doi  \
0  article-commentary  article  10.1017/S0007123400004890   
1    research-article  article  10.1017/S0007123409000684   
2    research-article  article                        NaN   
3    research-article  article  10.1017/S0007123400000405   
4    research-article  article  10.1017/S0007123415000587   

                                    id  \
0              ark://27927/pgh1fsbczbw   
1               ark://27927/pf1z0g3np0   
2  http://www.jstor.org/stable/2110667   
3              ark://27927/pgh1fsc4nsg   
4 

In [ ]:
# get the 'id' column from df_all
titles_in_df_all = df_all['id']

# Filter ngram DataFrame to only include rows where 'id' is in titles_in_df_all
ngram_filtered = ngram[ngram['id'].isin(titles_in_df_all)]

# Count the number of unigrams, bigrams, and trigrams
ngram_filtered = ngram_filtered[['id', 'unigramCount', 'bigramCount', 'trigramCount']]

# Group by 'id' and sum the counts
print(ngram_filtered.head())

# Convert ngram_filtered (Dask DataFrame) to pandas DataFrame
ngram_filtered_pandas = ngram_filtered.compute()

# Perform the merge with df_all (which is pandas)
df_all_merged = pd.merge(df_all, ngram_filtered_pandas[['id', 'unigramCount', 'bigramCount', 'trigramCount']], on='id', how='left')

# Display the merged DataFrame
print(df_all_merged.head())

                                    id  \
1               ark://27927/pf1z0g3np0   
2  http://www.jstor.org/stable/2110667   
4               ark://27927/phwjwkc9w4   
5               ark://27927/pghjm06ksf   
6              ark://27927/phz45krzm5c   

                                        unigramCount  \
1  {'May’s': 3, 'Many.5': 1, 'communication)': 1,...   
2  {'two-party': 1, 'informa-': 1, 'these': 5, 's...   
4  {'2003.': 8, 'New': 10, '1989–98,': 1, 'rule':...   
5  {'M.': 11, 'modelling.': 3, 'others': 1, 'expl...   
6  {'(2004,': 1, 'disgusted': 2, 'long': 2, 'phil...   

                                         bigramCount  \
1  {'guaranteeing that': 1, 'original situation.)...   
2  {'(2650) Net': 1, '"the least': 1, 'the looks'...   
4  {'eight times': 1, 'development.45 This': 1, '...   
5  {'represents the': 1, 'Everyday 55': 1, '349–6...   
6  {'even publicly,': 1, 'contests a': 1, 'will “...   

                                        trigramCount  
1  {'a majority-ru

In [30]:
df_all_merged.to_json('df_decade_ngram.jsonl', orient='records', lines=True)

In [ ]:
### word frequency in subcorpus by ngram
def clean_word(word):
    word = word.lower()  # lowercase
    word = re.sub(r'[\d]', '', word)  # remove digits
    word = re.sub(r'[^\w\s]', '', word)  # remove punctuation
    word = word.strip() 
    return word if len(word) > 1 else ""

# Generalized function to expand n-gram frequency
def expand_ngrams(df, ngram_type):
    rows = []
    
    # Determine the required n-gram length for filtering
    ngram_length = 1  # Default to unigram if no valid ngram type is found
    if ngram_type == 'bigramCount':
        ngram_length = 2
    elif ngram_type == 'trigramCount':
        ngram_length = 3

    for _, row in df.iterrows():
        year_range = row['year_range']
        publication = row['isPartOf']
        ngram_dict = row.get(ngram_type, {})  # Get the unigram, bigram, or trigram

        if isinstance(ngram_dict, dict):  
            for phrase, count in ngram_dict.items():
                # Clean each word in the n-gram (unigram, bigram, trigram)
                cleaned_phrase = " ".join([clean_word(w) for w in phrase.split()])

                # Filter out phrases that don't meet the expected length after cleaning
                if cleaned_phrase.strip() and len(cleaned_phrase.split()) == ngram_length:  # Ensure exact n-gram length
                    rows.append((year_range, publication, cleaned_phrase, count))
    
    return pd.DataFrame(rows, columns=['year_range', 'publication', 'ngram', 'ngram_count'])

# Example usage:
unigram_df = expand_ngrams(df_all_merged, 'unigramCount')
bigram_df = expand_ngrams(df_all_merged, 'bigramCount')
trigram_df = expand_ngrams(df_all_merged, 'trigramCount')

In [ ]:
# Function to process and save n-gram data
def process_and_save(df, ngram_type):
    # word frequency in subcorpus
    subcorpus_groups = df.groupby(['year_range', 'publication', 'ngram'], as_index=False).sum()

    # save to CSV
    for (year_range, publication), group in subcorpus_groups.groupby(['year_range', 'publication']):
        output_df = group[['ngram', 'ngram_count']].sort_values(by='ngram_count', ascending=False)
        filename = f"{ngram_type}_{year_range}_{publication}.csv"
        output_df.to_csv(filename, index=False)
        print(f"Saved: {filename}")

process_and_save(unigram_df, "unigram")
process_and_save(bigram_df, "bigram")
process_and_save(trigram_df, "trigram")

Saved: unigram_1971-1980_American Journal of Political Science.csv
Saved: unigram_1971-1980_British Journal of Political Science.csv
Saved: unigram_1981-1990_American Journal of Political Science.csv
Saved: unigram_1981-1990_British Journal of Political Science.csv
Saved: unigram_1991-2000_American Journal of Political Science.csv
Saved: unigram_1991-2000_British Journal of Political Science.csv
Saved: unigram_2001-2010_American Journal of Political Science.csv
Saved: unigram_2001-2010_British Journal of Political Science.csv
Saved: unigram_2011-2020_American Journal of Political Science.csv
Saved: unigram_2011-2020_British Journal of Political Science.csv
Saved: unigram_2021-2024_American Journal of Political Science.csv
Saved: unigram_2021-2024_British Journal of Political Science.csv
Saved: bigram_1971-1980_American Journal of Political Science.csv
Saved: bigram_1971-1980_British Journal of Political Science.csv
Saved: bigram_1981-1990_American Journal of Political Science.csv
Saved